Al momento hai:

Un indice TF-IDF sui profili testuali (text_profile) dei giocatori.

Uno strato “intelligente” di query (Step C) che:

interpreta ruolo (PG/SG/SF/PF/C),

riconosce skill (tiratore da 3, rimbalzista offensivo, difensore, playmaker, scorer…),

costruisce una query pesata e fa ranking con similarità coseno.

👉 Con lo Step D vogliamo:

Tenere questo ranking,

ma aggiungere un livello di spiegabilità:

per ogni giocatore top-N, dire perché è compatibile con la richiesta, usando:

ruolo,

statistiche (3P%, FT%, TRB, ORB, AST, DWS, PTS, …),

skill richieste nella query.

In [1]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice + dataset stats
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# Carico dataset completo con le stats per spiegazioni
full_df = pd.read_csv(DATA_CLEAN_PATH)
print("Dataset stats shape:", full_df.shape)

# Assumo che player_id sia l'indice originale usato in Step A
if "player_id" not in metadata_df.columns:
    # Se non c'è, lo ricreo coerente
    metadata_df["player_id"] = metadata_df.index.astype(int)

# ============================
# 2. Utility di base
# ============================

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

def safe_get(row, col, default=None):
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

# ============================
# 3. Stesse regole semantiche Step C
# ============================

ROLE_PATTERNS = {
    "PG": ["playmaker", "point guard", "regista", "portatore di palla"],
    "SG": ["guardia tiratrice", "guardia", "shooting guard"],
    "SF": ["ala piccola", "small forward"],
    "PF": ["ala grande", "power forward"],
    "C":  ["centro", "pivot"]
}

SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo"
    ],
    "reb_off": [
        "rimbalzista offensivo", "rimbalzo offensivo", "forte rimbalzista offensivo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "defense": [
        "difensore", "forte difensore", "buon difensore",
        "difesa", "difensiva", "specialista difensivo"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

SEMANTIC_EXPANSIONS = {
    "PG": "ruolo playmaker playmaker PG",
    "SG": "ruolo guardia tiratrice SG",
    "SF": "ruolo ala piccola SF",
    "PF": "ruolo ala grande PF",
    "C":  "ruolo centro C",

    "shooting_3": "ottimo tiratore da 3 punti tiro da 3 3P% buon tiratore da 3",
    "ft":         "buon tiratore di tiri liberi FT% tiri liberi eccellente tiratore di tiri liberi",
    "reb":        "forte rimbalzista rimbalzi buon rimbalzista",
    "reb_off":    "molto forte a rimbalzo offensivo rimbalzista offensivo rimbalzi offensivi",
    "playmaking": "ottimo playmaker e passatore buon passatore assist",
    "defense":    "difensore di alto livello buon difensore profilo difensivo interessante",
    "scorer":     "segna in media molti punti punti a partita realizzatore"
}

SEMANTIC_WEIGHTS = {
    "PG": 3,
    "SG": 3,
    "SF": 3,
    "PF": 3,
    "C":  3,

    "shooting_3": 4,
    "ft":         3,
    "reb":        3,
    "reb_off":    4,
    "playmaking": 3,
    "defense":    3,
    "scorer":     3
}

# ============================
# 4. Parsing query -> intenti
# ============================

def parse_query_intents(raw_query: str):
    q = normalize_text(raw_query)
    detected_roles = set()
    detected_skills = set()

    for role, patterns in ROLE_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_roles.add(role)
                break

    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    intents = {
        "normalized_query": q,
        "roles": sorted(list(detected_roles)),
        "skills": sorted(list(detected_skills))
    }
    return intents

def build_weighted_query(raw_query: str, intents: dict) -> str:
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted_parts = [base]

    for role in intents.get("roles", []):
        if role in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[role]
            w = SEMANTIC_WEIGHTS.get(role, 1)
            boosted_parts.append((" " + exp) * w)

    for skill in intents.get("skills", []):
        if skill in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[skill]
            w = SEMANTIC_WEIGHTS.get(skill, 1)
            boosted_parts.append((" " + exp) * w)

    weighted_query = " ".join(boosted_parts)
    return weighted_query.strip()

# ============================
# 5. Ricerca con query pesata (come Step C)
# ============================

def search_players_scout(query: str, top_k: int = 10):
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    intents = parse_query_intents(query)
    weighted_query = build_weighted_query(query, intents)

    query_vec = vectorizer.transform([weighted_query])
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()
    top_idx = np.argsort(sim_scores)[::-1][:top_k]

    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    results.attrs["intents"] = intents
    results.attrs["weighted_query"] = weighted_query
    return results

# ============================
# 6. Funzioni per spiegazioni
# ============================

# ============================
# 6. Funzioni per spiegazioni (VERSIONE CORRETTA)
# ============================

def format_percent_it(value, decimals=1):
    """
    Converte 0.384 -> '38,4%' e 38.4 -> '38,4%'.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    s = f"{v:.{decimals}f}"
    s = s.replace(".", ",")  # formattazione "italiana"
    return s + "%"

def format_number_it(value, decimals=1):
    """
    Formatta un numero con virgola italiana: 6.5 -> '6,5'
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except (TypeError, ValueError):
        return None
    s = f"{v:.{decimals}f}"
    return s.replace(".", ",")

def bucket_3p(pct):
    if pct is None or pd.isna(pct):
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucket_ft(pct):
    if pct is None or pd.isna(pct):
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    else:
        return "tiratore di liberi migliorabile"

def build_explanation(player_row, intents):
    """
    Costruisce una breve spiegazione del perché il giocatore matcha la query.
    Usa:
      - ruolo (Pos)
      - stats numeriche (3P%, FT%, TRB per partita, ORB per partita, AST, DWS, PTS)
      - skill richieste
    """
    reasons = []

    name = safe_get(player_row, "Player")
    if name:
        reasons.append(f"{name} è stato selezionato perché:")

    # --- Percentuali tiro ---
    p3 = safe_get(player_row, "3P%") or safe_get(player_row, "ThreeP")
    ft = safe_get(player_row, "FT%") or safe_get(player_row, "FT")

    skills = intents.get("skills", [])

    # Shooting da 3
    if "shooting_3" in skills and p3 is not None:
        desc_3p = bucket_3p(p3)
        p3_str = format_percent_it(p3)
        if desc_3p and p3_str:
            reasons.append(f"- {desc_3p} ({p3_str} da tre).")

    # Tiri liberi
    if "ft" in skills and ft is not None:
        desc_ft = bucket_ft(ft)
        ft_str = format_percent_it(ft)
        if desc_ft and ft_str:
            reasons.append(f"- {desc_ft} ({ft_str} ai liberi).")

    # --- Rimbalzi: usa per-partita se disponibile ---
    # Proviamo prima colonne "per game". Tu mi hai detto che i per-game sono in AST.1,
    # quindi la includo esplicitamente tra le possibilità.
    trb_pg = None

    for col in ["TRB_per_game", "TRB_pg", "TRB.1", "AST.1"]:
        if col in player_row.index and not pd.isna(player_row[col]):
            trb_pg = player_row[col]
            break

    # Se non trovo rimbalzi per partita, provo a derivarli da TRB totale / G
    if trb_pg is None:
        trb_total = safe_get(player_row, "TRB") or safe_get(player_row, "REB")
        games = safe_get(player_row, "G")
        try:
            if trb_total is not None and games not in (None, 0, np.nan):
                trb_pg = float(trb_total) / float(games)
        except:
            trb_pg = None

    # Rimbalzi totali offensivi per partita (se hai una colonna apposta)
    orb_pg = None
    for col in ["ORB_per_game", "ORB_pg"]:
        if col in player_row.index and not pd.isna(player_row[col]):
            orb_pg = player_row[col]
            break
    if orb_pg is None:
        # fallback: ORB totale / G
        orb_total = safe_get(player_row, "ORB")
        games = safe_get(player_row, "G")
        try:
            if orb_total is not None and games not in (None, 0, np.nan):
                orb_pg = float(orb_total) / float(games)
        except:
            orb_pg = None

    # Rimbalzi difensivi generali
    if "reb" in skills and trb_pg is not None:
        try:
            v = float(trb_pg)
            v_str = format_number_it(v)
            if v >= 9:
                reasons.append(f"- È un forte rimbalzista ({v_str} rimbalzi a partita).")
            elif v >= 6:
                reasons.append(f"- È un buon rimbalzista ({v_str} rimbalzi a partita).")
        except:
            pass

    # Rimbalzi offensivi
    if "reb_off" in skills and orb_pg is not None:
        try:
            v = float(orb_pg)
            v_str = format_number_it(v)
            if v >= 3:
                reasons.append(f"- Molto forte a rimbalzo offensivo ({v_str} rimbalzi offensivi a partita).")
            elif v >= 1.5:
                reasons.append(f"- Buon rimbalzista offensivo ({v_str} rimbalzi offensivi a partita).")
        except:
            pass

    # --- Playmaking ---
    ast = safe_get(player_row, "AST")  # qui di solito è AST per partita
    if "playmaking" in skills and ast is not None:
        try:
            v = float(ast)
            v_str = format_number_it(v)
            if v >= 7:
                reasons.append(f"- Ottimo playmaker e passatore ({v_str} assist a partita).")
            elif v >= 4:
                reasons.append(f"- Buon passatore ({v_str} assist a partita).")
        except:
            pass

    # --- Difesa ---
    dws = safe_get(player_row, "DWS") or safe_get(player_row, "Def_WS")
    stl = safe_get(player_row, "STL")
    blk = safe_get(player_row, "BLK")

    if "defense" in skills:
        added = False
        if dws is not None:
            try:
                v = float(dws)
                if v >= 3:
                    reasons.append("- Difensore di alto livello (ottimi indici difensivi).")
                    added = True
                elif v >= 1.5:
                    reasons.append("- Buon difensore (buoni indici difensivi).")
                    added = True
            except:
                pass
        if not added and (stl is not None or blk is not None):
            try:
                stl_v = float(stl) if stl is not None else 0.0
                blk_v = float(blk) if blk is not None else 0.0
                if stl_v >= 1.5 or blk_v >= 1.5:
                    reasons.append("- Profilo difensivo interessante (recuperi/stoppate).")
            except:
                pass

    # --- Scorer ---
    pts = safe_get(player_row, "PTS") or safe_get(player_row, "PTS_per_game")
    if "scorer" in skills and pts is not None:
        try:
            v = float(pts)
            v_str = format_number_it(v)
            if v >= 20:
                reasons.append(f"- Realizzatore di alto livello ({v_str} punti a partita).")
            elif v >= 15:
                reasons.append(f"- Buon realizzatore ({v_str} punti a partita).")
        except:
            pass

    # Fallback se non ho ragioni specifiche
    if len(reasons) <= 1:
        reasons.append("- Il suo profilo testuale è complessivamente molto simile alla descrizione richiesta.")

    explanation = "\n".join(reasons)
    return explanation

# ============================
# 7. Funzione principale: ranking + spiegazioni
# ============================

def rank_and_explain(query: str, top_k: int = 5):
    """
    - Usa search_players_scout per ottenere ranking in base al profilo testuale.
    - Fa merge con il dataset completo per avere le stats.
    - Costruisce una spiegazione per ogni giocatore.
    """
    base_results = search_players_scout(query, top_k=top_k)
    intents = base_results.attrs["intents"]

    # Merge con full_df sulle chiavi migliori che hai (player_id o Player)
    if "player_id" in base_results.columns and "player_id" in full_df.columns:
        merged = base_results.merge(full_df, on="player_id", suffixes=("", "_full"), how="left")
    else:
        merged = base_results.merge(full_df, on="Player", suffixes=("", "_full"), how="left")

    explanations = []
    for _, row in merged.iterrows():
        explanations.append(build_explanation(row, intents))

    merged["explanation"] = explanations
    return merged, intents

# ============================
# 8. Test
# ============================

test_query = "Cerco un tiratore con ottime percentuali da 3, buon FT%, forte rimbalzista offensivo e buon difensore."

results, intents = rank_and_explain(test_query, top_k=5)

print("Query originale:")
print(test_query)
print("\nIntenti interpretati:")
print(intents)

pd.set_option("display.max_colwidth", 200)
cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity", "explanation"] if c in results.columns]

print("\nTOP 5 con spiegazioni:")

#Stampo i risultati con spiegazione togliendo il carattere \n

results["explanation_one_line"] = results["explanation"].str.replace("\n", " ")

display(results[["Player", "explanation_one_line"]])




Indice caricato.
TF-IDF matrix shape: (8323, 11236)
Metadata shape: (8323, 6)
Dataset stats shape: (8323, 35)
Query originale:
Cerco un tiratore con ottime percentuali da 3, buon FT%, forte rimbalzista offensivo e buon difensore.

Intenti interpretati:
{'normalized_query': 'cerco un tiratore con ottime percentuali da 3, buon ft%, forte rimbalzista offensivo e buon difensore.', 'roles': [], 'skills': ['defense', 'ft', 'reb', 'reb_off', 'shooting_3']}

TOP 5 con spiegazioni:


,Player,explanation_one_line
0,Detlef Schrempf,"Detlef Schrempf è stato selezionato perché: - buon tiratore da 3 punti (38,4% da tre). - buon tiratore di tiri liberi (80,3% ai liberi). - È un buon rimbalzista (6,2 rimbalzi a partita)."
1,Carrick Felix,"Carrick Felix è stato selezionato perché: - ottimo tiratore da 3 punti (40,0% da tre). - buon tiratore di tiri liberi (75,0% ai liberi)."
2,Ayo Dosunmu,"Ayo Dosunmu è stato selezionato perché: - buon tiratore da 3 punti (36,1% da tre). - buon tiratore di tiri liberi (77,3% ai liberi)."
3,John Stockton,"John Stockton è stato selezionato perché: - buon tiratore da 3 punti (38,4% da tre). - buon tiratore di tiri liberi (82,6% ai liberi)."
4,D.J. Augustin,"D.J. Augustin è stato selezionato perché: - buon tiratore da 3 punti (38,1% da tre). - eccellente tiratore di tiri liberi (86,7% ai liberi)."


Ora il tuo sistema non fa solo:

“Ecco i giocatori simili alla query”.

ma fa:

“Ecco i giocatori simili e ti spiego perché ciascuno è in classifica.”

In pratica hai ottenuto:

Un ranking basato su IR + query pesata (Step C),

Un livello di explainable AI:

ruolo coerente con la richiesta,

statistiche compatibili con le skill richieste (3P%, FT%, TRB, ORB, AST, DWS, PTS),

frase in linguaggio naturale, pronta da mostrare a un talent scout / direttore sportivo.

Questo è perfetto per la presentazione: puoi dire che la Parte 2 non è solo un motore di ricerca, ma un vero assistente intelligente per lo scouting NBA, che non solo restituisce un ranking, ma giustifica anche le sue raccomandazioni. 🏀📊